# Tarea 1 Máquinas de Aprendizaje

## Ejercicio 1

a) Construcción del dataframe. La primera columna del dataframe original es redundante para la indexación, mientras que la columna $Train$ nos permite identificar cuales ejemplos serán parte del training set ($Train = T$) y del testing set ($Train = F$). Para explicitar qué ejemplos son del testing set se invierten los valores de verdad de dicha columna. Finalmente la columna ya utilizada se descarta para quedarnos con las columnas de predictores.

In [ ]:
import numpy as np
import pandas as pd

url = 'http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/prostate.data'
df = pd.read_csv(url, sep='\t', header=0)
# Remover columna con indices redundantes
df = df.drop('Unnamed: 0', axis=1)
# Obtener columna con la etiqueta Train y reemplazar valores booleanos. Estos ejemplos seran de entrenamiento
istrain_str = df['train']
istrain = np.asarray([True if s == 'T' else False for s in istrain_str])
# Listar como testing el resto de valores falsos de la columna anterior
istest = np.logical_not(istrain)
# Una vez procesado los datos, eliminar la columna train para almacenar los predictores relevantes
df = df.drop('train', axis=1)

b) Descripción del dataset. El dataset posee 9 atributos y 97 samples con valores enteros y reales.

In [ ]:
print df.shape
df.info()
df.describe()

c) Normalización de datos. Este preprocesamiento de los datos es importante pues las features originales pueden venir en distintas escalas por lo tanto nuestro algoritmo de aprendizaje no funcionará correctamente, por ejemplo al utilizar funciones objetivo que incluyan métricas, los datos con mayor rango tenderán a dominar sobre los de menor rango; por otra parte también podría darse el caso de que la convergencia obtenida sea lenta o imprecisa.

In [ ]:
from sklearn.preprocessing import StandardScaler
# Por defecto centra y escala la data.
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
#  Deseamos aprender a predecir el feature lpsa, por lo que la recuperamos del original
df_scaled['lpsa'] = df['lpsa']

d) Regresión lineal con Mínimos Cuadrados. En primer lugar extraemos la última columna de los datos, que corresponde al output $y$ de cada ejemplo. La nueva columna añadida corresponde al bias clásico...
El argumento pasado al constructor de LinearRegression indica que no se calculará intercepto para el modelo (ya lo hemos hecho a través de normalizar e ingresar la columna con bias 1)

In [ ]:
import sklearn.linear_model as lm
X = df_scaled.ix[:,:-1]
# Agregamos la columna de bias con 1
N = X.shape[0]
X.insert(X.shape[1], 'intercept', np.ones(N))
# Obtener los datos de output conocidos y extraer test & training set
y = df_scaled['lpsa']
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[istest]
ytest = y[istest]
linreg = lm.LinearRegression(fit_intercept = False)
linreg.fit(Xtrain, ytrain)